In [1]:
%load_ext autoreload
%matplotlib inline
%autoreload 2

In [2]:
import uproot
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import pickle
import h5py

In [39]:
sys.path.append("../../WatChMaL_analysis_copy")

from WatChMaL.analysis.multi_plot_utils import multi_disp_learn_hist, multi_compute_roc, multi_plot_roc, compute_roc, plot_roc
from WatChMaL.analysis.comparison_utils import multi_get_masked_data, multi_collapse_test_output, collapse_test_output
from WatChMaL.analysis.fitqun_comparison_utils import load_mu_fq_output, load_pion_fq_output, load_gamma_fq_output
from WatChMaL.analysis.performance_analysis_utils import remove_indices
from WatChMaL.analysis.performance_analysis_plot_utils import plot_fitqun_binned_performance, plot_momentum_binned_performance
from WatChMaL.analysis.performance_analysis_plot_utils import plot_true_momentum_binned_performance, plot_to_wall_binned_performance, plot_energy_binned_performance, plot_zenith_binned_performance, plot_azimuth_binned_performance
from WatChMaL.analysis.performance_analysis_plot_utils import plot_to_wall_binned_in_energy, plot_zenith_binned_in_azimuth, plot_azimuth_binned_in_zenith

In [4]:
############# define plotting params #############
c = plt.rcParams['axes.prop_cycle'].by_key()['color']
label_dict = {"$\gamma$":0, "$e$":1, "$\mu$":2, "$\pi 0$":3}
inverse_label_dict = {0:"$\gamma$", 1:"$e$", 2:"$\mu$", 3:"$\pi 0$"}
muon_softmax_index_dict = {"e/gamma":0, "mu":1}

<h1>Load fiTQun Results</h1>

In [7]:
# Retrieve flags
gamma_file_path = '/fast_scratch/WatChMaL/data/IWCD_mPMT_Short_gamma_E0to1000MeV_unif-pos-R400-y300cm_4pi-dir.fiTQun.root'
e_file_path     = '/fast_scratch/WatChMaL/data/IWCD_mPMT_Short_e-_E0to1000MeV_unif-pos-R400-y300cm_4pi-dir.fiTQun.root'
mu_file_path    = '/fast_scratch/WatChMaL/data/IWCD_mPMT_Short_mu-_E0to1000MeV_unif-pos-R400-y300cm_4pi-dir.fiTQun.root'
pion_file_path  = '/fast_scratch/WatChMaL/data/IWCD_mPMT_Short_pi0_E0to1000MeV_unif-pos-R400-y300cm_4pi-dir.fiTQun.root'

fq_mapping_path = '/home/zpatel/WatChMaL_analysis_copy/fitqun_comparison/fitqun_comparison_prep/prep_data/4_class_3M_fitqun_mapping.pickle'

with open('/home/zpatel/WatChMaL_analysis_copy/fitqun_comparison/fitqun_comparison_prep/prep_data/4_class_3M_fitqun_cuts.pickle', 'rb') as handle:
    cuts = pickle.load(handle)

<h1>Load ResNet Results</h1>

In [15]:
############# define run locations #############
short_locs = [#'/home/jtindall/WatChMaL/outputs/2021-04-06/13-38-38/outputs'
#               '/home/zpatel/WatChMaL/outputs/2021-05-31/09-44-17/outputs' # 4 class
#               '/home/zpatel/WatChMaL/outputs/2021-04-19/09-44-40/outputs'   # 2 class
                '/home/zpatel/WatChMaL/outputs/2021-06-07/09-32-54/outputs'
              ]

short_titles = ['Short Tank (4-class)', 'Short Tank (2-class)', 'Fitqun']

full_locs = []

full_titles = []

short_linecolor = [c[0] for _ in range(2)]
short_linestyle = ['--' for _ in range(2)]

full_linecolor = [c[1] for _ in full_locs]
full_linestyle = [':' for _ in full_locs]


In [16]:
############# load short tank run data #############
short_raw_output_softmax    = [np.load(loc + "/softmax.npy") for loc in short_locs]
short_raw_actual_labels     = [np.load(loc + "/labels.npy") for loc in short_locs]

In [17]:
locs      = short_locs + full_locs
titles    = short_titles + full_titles
linecolor = short_linecolor + full_linecolor
linestyle = short_linestyle + full_linestyle

<h1>Load h5 Data</h1>

In [18]:
# Import test events from h5 file
data_path = "/fast_scratch/WatChMaL/data/IWCD_mPMT_Short_emgp0_E0to1000MeV_digihits.h5"
data_file = h5py.File(data_path, "r")

print(data_file.keys())

energies   = np.array(data_file['energies'])
angles     = np.array(data_file['angles'])
positions  = np.array(data_file['positions'])
labels     = np.array(data_file['labels'])

<KeysViewHDF5 ['angles', 'energies', 'event_hits_index', 'event_ids', 'hit_charge', 'hit_pmt', 'hit_time', 'labels', 'positions', 'root_files', 'veto', 'veto2']>


In [19]:
idxs_path = '/fast_scratch/WatChMaL/data/IWCD_mPMT_Short_4_class_3M_emgp0_idxs.npz'
idxs = np.load(idxs_path, allow_pickle=True)

test_idxs  = idxs['test_idxs']

test_angles    = angles[test_idxs]
test_energies  = energies[test_idxs]
test_positions = positions[test_idxs]
test_labels    = labels[test_idxs]

In [20]:
d_to_wall = np.load('/home/zpatel/WatChMaL_analysis_copy/fitqun_comparison/fq_comparison_data/3M_d_to_wall.npz', allow_pickle=True)

test_d_to_wall  = d_to_wall['test_d_to_wall']

In [21]:
true_mom = np.load('/home/zpatel/WatChMaL_analysis_copy/fitqun_comparison/fq_comparison_data/3M_momenta.npz', allow_pickle=True)

test_true_mom  = true_mom['test_momenta']

<h1>Verify Indices</h1>

In [22]:
e_gamma_4_class_idxs = np.where((test_labels == 0) | (test_labels == 1))[0]

In [34]:
print(np.bincount(short_raw_actual_labels[0]))
# print(np.bincount(short_raw_actual_labels[1]))
print(np.bincount(test_labels))
print(np.bincount(test_labels[e_gamma_4_class_idxs]))

[1182377 1177966 1164139 1147267]
[1182377 1177966 1164139 1147267]
[1182377 1177966]
[1182377 1177966]


<h1>Generate Muon Rejection Based Preselection</h1>

<h2>Load 4-class with muons</h2>

In [35]:
muon_softmax_index_dict = {"e/gamma":0, "mu":1}

In [36]:
muon_cut_to_apply = cuts['fq_comparison']

muon_raw_short_softmax = short_raw_output_softmax[0]

muon_filtered_short_softmax  = remove_indices(short_raw_output_softmax[0], muon_cut_to_apply)
muon_filtered_labels         = remove_indices(short_raw_actual_labels[0], muon_cut_to_apply)

muon_collapsed_class_scores, muon_collapsed_class_labels = collapse_test_output(muon_filtered_short_softmax, muon_filtered_labels, label_dict, ignore_type='$\gamma$')

muon_collapsed_class_scores = muon_collapsed_class_scores[:,1:]
muon_collapsed_class_labels = muon_collapsed_class_labels - 1 

<h2>Cuts</h2>

<p>As the electron neutrino 
ux is a small component of the total neutrino 
ux, electron neutrino interaction events make up only 1.5% of all interaction events in the detector. In order to make precise measurements of electron neutrino cross-sections it is necessary to select a sample of events enriched in electron neutrino interactions. These events must be selected based upon the properties of the event as reconstructed by the detector.</p>

<h2> Compute Threshold </h2>

Choose a threshold as close to fpr_fixed_point = 0.005 as possible. To do this, compute index of min(fprs - 0.005) to find the false positive rate closest to the fixed point of 0.005. The index of this FPR will provide the threshold.

In [40]:
fpr_fixed_point = 0.005
fprs, tprs, thrs = compute_roc(muon_collapsed_class_scores, muon_collapsed_class_labels, 
                            true_label=muon_softmax_index_dict["e/gamma"], 
                            false_label=muon_softmax_index_dict["mu"],
                            normalize=True)

operating_point_thr = thrs[np.argmin(np.abs(fprs - fpr_fixed_point))]

operating_point_cut = ((muon_raw_short_softmax[:, 0] + muon_raw_short_softmax[:, 1])/(muon_raw_short_softmax[:, 0] + muon_raw_short_softmax[:, 1] + muon_raw_short_softmax[:, 2])) < operating_point_thr

print(tprs[np.argmin(np.abs(fprs - fpr_fixed_point))])
print(fprs[np.argmin(np.abs(fprs - fpr_fixed_point))])

0.9737778274155542
0.004998784573403542


In [ ]:
plot_roc()

In [46]:
operating_point_thr

0.9070898680085112

In [44]:
np.argmin(np.abs(fprs - fpr_fixed_point))

4567

In [45]:
# Verify cut working
comp_idxs = np.where((muon_filtered_labels == 1 ) | (muon_filtered_labels == 2))[0]

scores = ((muon_filtered_short_softmax[:, 0] + muon_filtered_short_softmax[:, 1])/(muon_filtered_short_softmax[:, 0] + muon_filtered_short_softmax[:, 1] + muon_filtered_short_softmax[:, 2]))

pred_pos_idxs = np.where( scores[comp_idxs] >= operating_point_thr)[0]
pred_neg_idxs = np.where( scores[comp_idxs] < operating_point_thr)[0]

true_label, false_label = '$e$','$\mu$'
fp = np.where(muon_filtered_labels[comp_idxs[pred_pos_idxs]] == label_dict[false_label])[0].shape[0]
tp = np.where(muon_filtered_labels[comp_idxs[pred_pos_idxs]] == label_dict[true_label] )[0].shape[0]
fn = np.where(muon_filtered_labels[comp_idxs[pred_neg_idxs]] == label_dict[true_label] )[0].shape[0]
tn = np.where(muon_filtered_labels[comp_idxs[pred_neg_idxs]] == label_dict[false_label])[0].shape[0]

# TODO: division by zero problem
efficiency = performance = tp/(tp + fn + 1e-10)
fpr = performance = fp/(fp + tn + 1e-10)

print(efficiency)
print(fpr)

0.9737778274155541
0.004998784573403541
